# 1.0 Setup

In [1]:
!pip install -e "/content/drive/My Drive/Colab Notebooks/Applications of Machine Learning/tic_tac_toe_gym_env/"

Obtaining file:///content/drive/My%20Drive/Colab%20Notebooks/Applications%20of%20Machine%20Learning/tic_tac_toe_gym_env
  Found existing installation: gym-tictactoe 0.0.1
    Can't uninstall 'gym-tictactoe'. No files were found to uninstall.
  Running setup.py develop for gym-tictactoe


In [0]:
import numpy as np
import gym
import gym_tictactoe
import random
from tqdm import tqdm

# 2.0 Create Q-Table

In [3]:
env = gym.make('tictactoe-v0')
env.render()
q_table = {}

 | | 
-+-+-
 | | 
-+-+-
 | | 



In [0]:
action_dictionary = {
    0 : (0,0),
    1 : (0,1),
    2 : (0,2),
    3 : (1,0),
    4 : (1,1),
    5 : (1,2),
    6 : (2,0),
    7 : (2,1),
    8 : (2,2)
}

In [0]:
def hash_state(state):
  state_string = state.tostring()
  hash_value = hash(state_string)

  if hash_value not in q_table:
    q_table[hash_value] = np.full(9,0)

  return hash_value

In [0]:
def get_q_values(state):
  return q_table[hash_state(state)]

In [0]:
def get_action(code):
  return action_dictionary[code]

In [0]:
def get_action_code(action):
  for key, value in action_dictionary.items():
    if value == action:
      return key

In [0]:
def play_random_turn(state):
  valid_moves = np.where(state == 0)
  moves = [list(move) for move in zip(valid_moves[0], valid_moves[1])]
  moves_list = []
  for i in moves:
    move = tuple(i)
    moves_list.append(move)
  move = random.choice(moves_list)
  return move

# Testing

In [0]:
state = env.reset()
q_table = {}

In [11]:
q_table[hash_state(state)] = np.array([2,3,1,0,4,5,6,7,9])
q_table

{-8853504689288459033: array([2, 3, 1, 0, 4, 5, 6, 7, 9])}

In [0]:
step = 3

if (step % 2) == 0:
  print('true')
else:
  print('false')

false


In [0]:
def _game_over(state):
        """ checks if the game is over and returns the winning player or -1 if there is no winner"""
        for i in range(len(state)):
            if state[i, 0] != 0:
                if all([x == state[i, 0] for x in state[i]]):
                    return state[i, 0]
            if state[0, i] != 0:
                if all([x == state[0, i] for x in state[:, i]]):
                    return state[0, i]

        if state[0, 0] != 0:
            diag = np.diagonal(state)
            if all(x == diag[0] for x in diag):
                return state[0, 0]
        if state[0, 2] != 0:
            invdiag = np.diagonal(np.fliplr(state))
            if all(x == invdiag[0] for x in invdiag):
                return state[0, 2]
        if np.count_nonzero(state) == 9:
            return -1
        return False

        pass

In [16]:
state, reward, done = env.step((0,0))
state, reward, done

(array([[ 1.,  1.,  1.],
        [ 0.,  0., -1.],
        [ 0., -1.,  0.]]), 10, True)

In [15]:
move = play_random_turn(state)
env.step(move)

(array([[ 0.,  1.,  1.],
        [ 0.,  0., -1.],
        [ 0., -1.,  0.]]), 1, False)

In [17]:
result = _game_over(state)
result

1.0

# 3.0 Train Q-Table

In [0]:
total_episodes = 50000        # Total episodes
total_test_episodes = 100
max_steps = 9                # Max steps per episode

learning_rate = 0.7           # Learning rate
gamma = 0.95                 # Discount rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.01             # Exponential decay rate for exploration probability

In [11]:
%%time
for episode in tqdm(range(total_episodes)):
  state = env.reset()

  q_table[hash_state(state)] = np.full(9, 0)

  done = False
  
  for step in range(max_steps):

    if (step % 2) == 0:
      exploitation = random.uniform(0,1)

      if exploitation > epsilon:
        action = get_action(np.argmax(get_q_values(state)))

      else:
        action = tuple(env.action_space.sample())
         
      new_state, reward, done = env.step(action)

      q_table[hash_state(state)][get_action_code(action)] = q_table[hash_state(state)][get_action_code(action)] + learning_rate * (reward + gamma * np.max(q_table[hash_state(new_state)]) - q_table[hash_state(state)][get_action_code(action)])

    else:
      action = tuple(env.action_space.sample())

      new_state, reward, done = env.step(action)

    state = new_state
    
    if done:
      break
  
  episode += 1
 
  epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)

100%|██████████| 50000/50000 [00:16<00:00, 2968.94it/s]

CPU times: user 16.8 s, sys: 175 ms, total: 17 s
Wall time: 16.9 s


In [12]:
q_table

{-9086207524387291276: array([-14, -14, -14, -14, -14, -14, -14,   0,   0]),
 -9073688547717811431: array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 -9068847278550859274: array([-14, -14, -14,   0,   0,   0,   0,   0,   0]),
 -9067104890122737154: array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 -9051791097838128447: array([-14, -14,   0,   0,   0,   0,   0,   0,   0]),
 -9000367172614488785: array([-14, -14,   0,   0,   0,   0,   0,   0,   0]),
 -8972318118848914476: array([-14, -14, -14, -14, -14,   0,   0,   0,   0]),
 -8968978122988455609: array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 -8948216802870850253: array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 -8869293973046123243: array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 -8838904644674735831: array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 -8838260407121975751: array([-14,   0,   0,   0,   0,   0,   0,   0,   0]),
 -8815164907469697070: array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 -8745064295642575501: array([-14,   0,   0,   0,   0,   0, -14,   0,   0]),
 -8706443813246619014: array([0, 0, 0, 0, 0, 0, 

# 4.0 Play Tic-Tac-Toe

In [13]:
%%time
env.reset()
rewards = []
wins = 0
for episode in tqdm(range(total_test_episodes)):
  state = env.reset()
  step = 0
  done = False
  total_reward = 0
  print("**********************************************************")
  print("EPISODE: ", episode)

  for step in range(max_steps):
    
    if (step % 2) == 0:
      action = get_action(np.argmax(get_q_values(state)))

      new_state, reward, done = env.step(action)

      total_reward += reward

    else:
      action = tuple(env.action_space.sample())

      new_state, reward, done = env.step(action)

      total_reward += reward
    
    if done:
      rewards.append(total_reward)
      print(state)
      print("Score: ", total_reward)
      break
    
    state = new_state

100%|██████████| 100/100 [00:00<00:00, 636.96it/s]

**********************************************************
EPISODE:  0
[[ 1.  1.  0.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]]
Score:  -17
**********************************************************
EPISODE:  1
[[ 1.  1. -1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]]
Score:  -15
**********************************************************
EPISODE:  2
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0. -1. -1.]]
Score:  14
**********************************************************
EPISODE:  3
[[ 1. -1.  1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]]
Score:  -15
**********************************************************
EPISODE:  4
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Score:  -19
**********************************************************
EPISODE:  5
[[ 1.  1.  0.]
 [ 0.  0.  0.]
 [-1.  0.  0.]]
Score:  -17
**********************************************************
EPISODE:  6
[[ 1.  1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]
Score:  -17
**********************************************************
EPISODE:  7
[[ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. 

In [19]:
wins = 0
for score in rewards:
  if score > 10:
    wins+=1

print("Win Percentage: " + str(wins/100 *100) + "%")

Win Percentage: 39.0%
